In [1]:
import numpy as np
import pandas as pd

In [2]:
def objfun(s):
    
    """ 
    This function calculates the objective function value

    Parameters: 
        s (array): Candidate solution in 1x10 vector form [s1,s2,s3,s3,s5,s6,s7,s8,s9,s10]

    Returns: 
        z (int): objective function value
    """
    
    z = 8*s[0] + 12*s[1] + 9*s[2] + 14*s[3] + 16*s[4] + 10*s[5] + 6*s[6] + 7*s[7] + 11*s[8] + 13*s[9]
    
    return z    

In [3]:
def check_feasibility(s):
    
    """ 
    This function checks whether the candidate solution is feasible according to given constraint the objective function value

    Parameters: 
        s (array): Candidate solution in 1x10 vector form [s1,s2,s3,s3,s5,s6,s7,s8,s9,s10]

    Returns: 
        constraint_flag (bool): The flag that indicates whether the candidate solution is feasible. 
                                If it is, then constraint_flag = True, else False
    """
    
    # Calculate constraint value
    const = 3*s[0] + 2*s[1] + s[2] + 4*s[3] + 3*s[4] + 3*s[5] + s[6] + 2*s[7] + 2*s[8] + 5*s[9]
    
    # Check whether the candidate solution is valid subject to problem constraint, return True is yes and False if no
    if const > 12:
        constraint_flag = False
    else:
        constraint_flag = True
    
    return constraint_flag   

In [4]:
def generate_rand_init_cond(n):
    
    """ 
    This function generates a random initial condition of vector size 1xn, choosing either 0 or 1 randomly

    Parameters: 
        n (int): Length of candidate solution, equal to the number of decision variables

    Returns: 
        s (array): Initial candidate solution of size 1xn
    """
    
    # Check whether the initial condition satisfies the problem constraint(s) and is feasible
    # Generate new initial conditions until problem constraint(s) are satisfied
    flag = False
    while flag == False:
        s = np.random.choice([0, 1], size=(n,))
        flag = check_feasibility(s)
    
    return s

In [5]:
def intial_cond():

    """ 
    This function allows the user to determine whether they would prefer to randomly 
    generate an initial solution or enter their own

    Parameters: None

    Returns: 
        s (array): Initial candidate solution of size 1xn
    """
    
    # Prompt user to indicate whether they would prefer a random or user-defined initial condition
    print('Random (R) or User defined (U) initial condition? (Enter R or U):')
    
    sol_input = input()

    if sol_input == 'R':

        # number of decision variables as input 
        n = int(input('Number of decision variables (Enter an integer):'))

        # Generate randome initial condition
        s = generate_rand_init_cond(n)

    elif sol_input == 'U':

        # creating an empty list for initial solution to be appended to
        s_list = []

        # number of decision variables as input 
        n = int(input('Number of decision variables (Enter an integer):')) 

        print('Enter each element of the initial condition')

        # Iterate over entire initial condition to generate initial condition array, appending each user input boolean value
        for i in range(0, n): 
            ele = int(input())
            s_list.append(ele)
            
        # Convert list to Numpy array for use in other functions
        s = np.array(s_list)

    else:
        print('Invalid selection')
        
    return s

In [6]:
def bit_comp(s,i):

    """ 
    This function performs a single-bit complement of a binary array at a given position (i)

    Parameters:
        s (array): current candidate solution of size 1xn
        i (int): position at which the bit should be complemented

    Returns: 
        s_new (array): candidate solution after single-bit complement
    """
    
    s_new = np.copy(s)
    if s_new[i] == 0: 
        s_new[i] = 1
    else:
        s_new[i] = 0
    
    return s_new

In [7]:
def format_dataframe(data_store,k):

    """ 
    This function formats the output DataFrame from the local search for presentation in a report

    Parameters:
        data_store (Pandas DataFrame): record of all data from the local search with the following columns: 
        ['t', 's(t)', 'z', 'Neighbour', 'Bit', 'New_z']

    Returns: 
        data_store_formatted (Pandas DataFrame): record of all data from the local search in the correct format
    """
    
    # Iterate over all records
    for i in range(k):
    
        # Access the Neighbour array and assign to a
        a = data_store.loc[i,['Neighbour']][0]

        # Convert array to string in preparation for groupby
        data_store.loc[i,['Neighbour']] = np.array2string(a, separator=', ')

        # Access the s(t) array and assign to b
        b = data_store.loc[i,['s(t)']][0]

        # Convert array to string in preparation for groupby
        data_store.loc[i,['s(t)']] = np.array2string(b, separator=', ')
    
    # Perform groupby operation to generate the desired output format
    data_store_formatted = data_store.groupby(['t', 'z','s(t)', 'Neighbour','Bit','New_z'], as_index=True, sort=False).sum()    
    
    return data_store_formatted

In [8]:
def main():

    # Generate initial condition based on random generation or user input 
    s = intial_cond()

    # Declare global iteration counter representing the index of the dataframe over all search iterations 
    # (used to address and append to dataframe locations)
    # Declare search iteration counter and stop flag
    k = 0
    t = 1
    stop_flag = False

    # Create dataframe to store search data
    data_store = pd.DataFrame(columns=['t', 's(t)', 'z', 'Neighbour', 'Bit', 'New_z'])

    # Continue search iterations while the stopping criteria has not been reached
    while stop_flag == False:

        # Create dataframe to store current iteration objective function values and neighbours
        z_list = []

        # Calculate current objective function value
        z = objfun(s)

        # Perform single-bit complement for each bit in candidate solution, iterating over each bit in the solution vector
        # Check objective function value and store in dataframe (data_store) together with the corresponding bit position 
        for i in range(len(s)):

            # Store current data best solution data in dataframe
            data_store.loc[k,'t'] = t
            data_store.loc[k,'s(t)'] = s
            data_store.loc[k,'z'] = z

            # Perform bit complement on current bit position
            s_new = bit_comp(s,i)

            # Calculate objective function value
            z_new = objfun(s_new)

            # Check solution feasibility
            feasibility_flag = check_feasibility(s_new)

            # Set z_new to 'Infeasible' if candidate solution does not satisfy problem constraints
            if feasibility_flag == False:
                z_new = 'Infeasible'
            else:
                z_list.append(z_new)

            # Store new neighbour and objective function value in dataframe
            data_store.loc[k,'Neighbour'] = s_new
            data_store.loc[k,'Bit'] = i
            data_store.loc[k,'New_z'] = z_new

            # Increase global iteration counter
            k = k + 1

        # If current iteration produces no better objective function value, stopping condition is met. Set stop_flag = True
        if max(z_list) <= z:
            stop_flag = True

        # Else set new best solution to solution with best improvement
        else:

            # Isolate instances within current iteration only
            current_iter_data = data_store[data_store['t'] == t]

            # Convert New_z column to numeric in order to ignore 'Infeasible' strings when finding best solution index
            s_index = pd.to_numeric(current_iter_data['New_z'], errors='coerce').idxmax()

            # s now becomes the best solution of this iteration
            s = data_store.loc[s_index,['Neighbour']][0]

        # Increase search iteration counter
        t = t + 1

    # Format and return data_store DataFrame
    return format_dataframe(data_store,k)

In [11]:
main()

Random (R) or User defined (U) initial condition? (Enter R or U):


 U
Number of decision variables (Enter an integer): 10


Enter each element of the initial condition


 0
 0
 0
 0
 0
 0
 0
 0
 1
 0


Empty DataFrame
Columns: []
Index: [(1, 11, [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 0, 0, 0, 0, 1, 0], 0, 19), (1, 11, [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 0, 0, 1, 0], 1, 23), (1, 11, [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 1, 0, 0, 0, 0, 0, 1, 0], 2, 20), (1, 11, [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 1, 0, 0, 0, 0, 1, 0], 3, 25), (1, 11, [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 1, 0, 0, 0, 1, 0], 4, 27), (1, 11, [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1, 0, 0, 1, 0], 5, 21), (1, 11, [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 1, 0, 1, 0], 6, 17), (1, 11, [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 1, 1, 0], 7, 18), (1, 11, [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 8, 0), (1, 11, [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1], 9, 24), (2, 27, [0, 0, 0, 0, 1, 0, 0, 0, 1, 0], [1, 0, 0, 0, 1, 0, 0, 0, 1, 0], 0, 35), (2, 27, [0, 0, 0, 0, 1, 0, 0, 0, 1, 0], [0, 1, 0, 0, 1, 0, 0, 0, 1, 0], 1, 39), (2, 27, [0, 0, 0, 0, 1, 0, 0, 0, 1, 0], [0, 0, 1, 0, 1, 0, 0, 0, 1, 0], 2, 36), (2, 27, [0, 0, 0, 0, 1, 0, 0, 0, 1, 0], [0, 0, 0, 1, 1, 0, 0, 0, 1, 0], 3, 41), (2, 27, [0, 0, 0, 0, 1, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], 4, 11), (2, 27, [0, 0, 0, 0, 1, 0, 0, 0, 1, 0], [0, 0, 0, 0, 1, 1, 0, 0, 1, 0], 5, 37), (2, 27, [0, 0, 0, 0, 1, 0, 0, 0, 1, 0], [0, 0, 0, 0, 1, 0, 1, 0, 1, 0], 6, 33), (2, 27, [0, 0, 0, 0, 1, 0, 0, 0, 1, 0], [0, 0, 0, 0, 1, 0, 0, 1, 1, 0], 7, 34), (2, 27, [0, 0, 0, 0, 1, 0, 0, 0, 1, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0], 8, 16), (2, 27, [0, 0, 0, 0, 1, 0, 0, 0, 1, 0], [0, 0, 0, 0, 1, 0, 0, 0, 1, 1], 9, 40), (3, 41, [0, 0, 0, 1, 1, 0, 0, 0, 1, 0], [1, 0, 0, 1, 1, 0, 0, 0, 1, 0], 0, 49), (3, 41, [0, 0, 0, 1, 1, 0, 0, 0, 1, 0], [0, 1, 0, 1, 1, 0, 0, 0, 1, 0], 1, 53), (3, 41, [0, 0, 0, 1, 1, 0, 0, 0, 1, 0], [0, 0, 1, 1, 1, 0, 0, 0, 1, 0], 2, 50), (3, 41, [0, 0, 0, 1, 1, 0, 0, 0, 1, 0], [0, 0, 0, 0, 1, 0, 0, 0, 1, 0], 3, 27), (3, 41, [0, 0, 0, 1, 1, 0, 0, 0, 1, 0], [0, 0, 0, 1, 0, 0, 0, 0, 1, 0], 4, 25), (3, 41, [0, 0, 0, 1, 1, 0, 0, 0, 1, 0], [0, 0, 0, 1, 1, 1, 0, 0, 1, 0], 5, 51), (3, 41, [0, 0, 0, 1, 1, 0, 0, 0, 1, 0], [0, 0, 0, 1, 1, 0, 1, 0, 1, 0], 6, 47), (3, 41, [0, 0, 0, 1, 1, 0, 0, 0, 1, 0], [0, 0, 0, 1, 1, 0, 0, 1, 1, 0], 7, 48), (3, 41, [0, 0, 0, 1, 1, 0, 0, 0, 1, 0], [0, 0, 0, 1, 1, 0, 0, 0, 0, 0], 8, 30), (3, 41, [0, 0, 0, 1, 1, 0, 0, 0, 1, 0], [0, 0, 0, 1, 1, 0, 0, 0, 1, 1], 9, Infeasible), (4, 53, [0, 1, 0, 1, 1, 0, 0, 0, 1, 0], [1, 1, 0, 1, 1, 0, 0, 0, 1, 0], 0, Infeasible), (4, 53, [0, 1, 0, 1, 1, 0, 0, 0, 1, 0], [0, 0, 0, 1, 1, 0, 0, 0, 1, 0], 1, 41), (4, 53, [0, 1, 0, 1, 1, 0, 0, 0, 1, 0], [0, 1, 1, 1, 1, 0, 0, 0, 1, 0], 2, 62), (4, 53, [0, 1, 0, 1, 1, 0, 0, 0, 1, 0], [0, 1, 0, 0, 1, 0, 0, 0, 1, 0], 3, 39), (4, 53, [0, 1, 0, 1, 1, 0, 0, 0, 1, 0], [0, 1, 0, 1, 0, 0, 0, 0, 1, 0], 4, 37), (4, 53, [0, 1, 0, 1, 1, 0, 0, 0, 1, 0], [0, 1, 0, 1, 1, 1, 0, 0, 1, 0], 5, Infeasible), (4, 53, [0, 1, 0, 1, 1, 0, 0, 0, 1, 0], [0, 1, 0, 1, 1, 0, 1, 0, 1, 0], 6, 59), (4, 53, [0, 1, 0, 1, 1, 0, 0, 0, 1, 0], [0, 1, 0, 1, 1, 0, 0, 1, 1, 0], 7, Infeasible), (4, 53, [0, 1, 0, 1, 1, 0, 0, 0, 1, 0], [0, 1, 0, 1, 1, 0, 0, 0, 0, 0], 8, 42), (4, 53, [0, 1, 0, 1, 1, 0, 0, 0, 1, 0], [0, 1, 0, 1, 1, 0, 0, 0, 1, 1], 9, Infeasible), (5, 62, [0, 1, 1, 1, 1, 0, 0, 0, 1, 0], [1, 1, 1, 1, 1, 0, 0, 0, 1, 0], 0, Infeasible), (5, 62, [0, 1, 1, 1, 1, 0, 0, 0, 1, 0], [0, 0, 1, 1, 1, 0, 0, 0, 1, 0], 1, 50), (5, 62, [0, 1, 1, 1, 1, 0, 0, 0, 1, 0], [0, 1, 0, 1, 1, 0, 0, 0, 1, 0], 2, 53), (5, 62, [0, 1, 1, 1, 1, 0, 0, 0, 1, 0], [0, 1, 1, 0, 1, 0, 0, 0, 1, 0], 3, 48), (5, 62, [0, 1, 1, 1, 1, 0, 0, 0, 1, 0], [0, 1, 1, 1, 0, 0, 0, 0, 1, 0], 4, 46), (5, 62, [0, 1, 1, 1, 1, 0, 0, 0, 1, 0], [0, 1, 1, 1, 1, 1, 0, 0, 1, 0], 5, Infeasible), (5, 62, [0, 1, 1, 1, 1, 0, 0, 0, 1, 0], [0, 1, 1, 1, 1, 0, 1, 0, 1, 0], 6, Infeasible), (5, 62, [0, 1, 1, 1, 1, 0, 0, 0, 1, 0], [0, 1, 1, 1, 1, 0, 0, 1, 1, 0], 7, Infeasible), (5, 62, [0, 1, 1, 1, 1, 0, 0, 0, 1, 0], [0, 1, 1, 1, 

In [12]:
main()

Random (R) or User defined (U) initial condition? (Enter R or U):


 U
Number of decision variables (Enter an integer): 10


Enter each element of the initial condition


 0
 1
 1
 1
 0
 1
 0
 1
 0
 0


Empty DataFrame
Columns: []
Index: [(1, 52, [0, 1, 1, 1, 0, 1, 0, 1, 0, 0], [1, 1, 1, 1, 0, 1, 0, 1, 0, 0], 0, Infeasible), (1, 52, [0, 1, 1, 1, 0, 1, 0, 1, 0, 0], [0, 0, 1, 1, 0, 1, 0, 1, 0, 0], 1, 40), (1, 52, [0, 1, 1, 1, 0, 1, 0, 1, 0, 0], [0, 1, 0, 1, 0, 1, 0, 1, 0, 0], 2, 43), (1, 52, [0, 1, 1, 1, 0, 1, 0, 1, 0, 0], [0, 1, 1, 0, 0, 1, 0, 1, 0, 0], 3, 38), (1, 52, [0, 1, 1, 1, 0, 1, 0, 1, 0, 0], [0, 1, 1, 1, 1, 1, 0, 1, 0, 0], 4, Infeasible), (1, 52, [0, 1, 1, 1, 0, 1, 0, 1, 0, 0], [0, 1, 1, 1, 0, 0, 0, 1, 0, 0], 5, 42), (1, 52, [0, 1, 1, 1, 0, 1, 0, 1, 0, 0], [0, 1, 1, 1, 0, 1, 1, 1, 0, 0], 6, Infeasible), (1, 52, [0, 1, 1, 1, 0, 1, 0, 1, 0, 0], [0, 1, 1, 1, 0, 1, 0, 0, 0, 0], 7, 45), (1, 52, [0, 1, 1, 1, 0, 1, 0, 1, 0, 0], [0, 1, 1, 1, 0, 1, 0, 1, 1, 0], 8, Infeasible), (1, 52, [0, 1, 1, 1, 0, 1, 0, 1, 0, 0], [0, 1, 1, 1, 0, 1, 0, 1, 0, 1], 9, Infeasible)]